# Cryosphere model Comparison tool (CmCt) --- Calving

The CmCt Calving tool compares user uploaded ice sheet model to satellite calving data, derived from the   satellite missions. The CmCt reprojects modeled ice sheet mass change into calving space and performs a direct quantitative comparison, calculating model-observation discrepancies, therefore bringing the ice sheet model and calving data to the same spatial resolution for comparison.

## Calving dataset description

For comparison to the user's model, the tool uses the NASA GSFC GRACE mascon product, which can be downloaded from [here](https://earth.gsfc.nasa.gov/geo/data/grace-mascons/). Future developments will include adding the functionality to automatically check the product distribution website for an updated dataset.

## Input data requirements

The input ice sheet model needs to be provided as a netCDF file. The user may upload a single input file that includes multiple years; future enhancements to this tool will allow users to upload model ensembles.

There are several requirements for the comparison:

### `Lithk` variable

The CmCt Grace Mascon tool expects the uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](https://theghub.org/groups/ismip6/wiki). 

### Date range

The NASA GSFC GRACE mascon dataset starts on 04/2002 and is updated ~quarterly. The user can select start and end dates within this span as part of the setup for the tool.

In [2]:
## Import modules
import os,sys
import numpy as np
import cftime 

# Add the directory containing 'cmct' to the Python path
# Navigate two levels up to reach main CmCt dir
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

# Import utilities for this comparison
sys.path.insert(0,cmct_dir)
from cmct.time_utils import check_datarange
from cmct.calving import *
from cmct.gravimetry import *
from cmct.projection import *


### Finding the Dataset

In [3]:
# Ice sheet
loc = 'GIS' # 'GIS' or 'AIS'

# Set time range for comparison
start_date = '2006-01-01' 
end_date = '2014-12-31' 

# Set the observation data dir path
obs_filename = cmct_dir + '/data/calving/sftgif_GIS_JPL_ISSM_historical.nc'


# LEGACY Gravimetry data paths

# # Set the model data dir path
# if loc == "GIS":
#     # Greenland
#     nc_filename = cmct_dir + '/test/gris_dmdt_filt.nc'
# elif loc == "AIS":    
#     # Antartica
#     nc_filename = cmct_dir + '/test/ais_dmdt_grounded_filt.nc'

# # Set the output plot file names
# plot_filename = os.path.splitext(os.path.basename(nc_filename))[0] + '_calving_comp'
# output_plot_filename = cmct_dir + '/notebooks/Calving/'+ plot_filename + '.png'
# output_netcdf_filename = cmct_dir + '/notebooks/Calving/'+ plot_filename + '.nc'


In [4]:
# Check if observation file exist
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# # Check if model file exist
# if not os.path.exists(nc_filename):
#     raise FileNotFoundError(f"Model file not found: {nc_filename}")


## Set polar sterographic projection definition
Currently, the CmCt expects the model data to be provided in a specific polar stereographic projection. This is hardcoded in the notebook. As a future enhancement, the projection can be obtained from the loaded model.

In [5]:
polar_stereographic = set_projection(loc)

## Process observation

In [ ]:
print(obs_filename)
gsfc = load_gsfc_calving(obs_filename)
print(type(gsfc))
# print(gsfc.print_info())  # Print the gsfc object to see its contents
print(vars(gsfc)) 
  # Print the attributes of the gsfc object

/Users/aditya_pachpande/Documents/GitHub/CmCt/data/calving/sftgif_GIS_JPL_ISSM_historical.nc
<class 'cmct.calving.GSFCcalving'>
{'ds': <xarray.Dataset>
Dimensions:   (y: 577, x: 337, nv4: 4, time: 36)
Coordinates:
    lon       (y, x) float64 ...
    lat       (y, x) float64 ...
  * time      (time) object 1980-01-01 00:00:00 ... 2015-01-01 00:00:00
  * x         (x) float64 -7.2e+05 -7.15e+05 -7.1e+05 ... 9.55e+05 9.6e+05
  * y         (y) float64 -3.45e+06 -3.445e+06 -3.44e+06 ... -5.75e+05 -5.7e+05
Dimensions without coordinates: nv4
Data variables:
    lon_bnds  (y, x, nv4) float64 ...
    lat_bnds  (y, x, nv4) float64 ...
    sftgif    (time, y, x) float64 ...
Attributes:
    Conventions:  CF-1.6
    title:        ISMIP6 Projections Greenland model output
    institution:  NASA Jet Propulsion Laboratory, Pasadena, USA
    source:       ISSM
    references:   https://doi.org/10.5194/tc-14-3071-2020
    contact:      Name = Helene Seroussi, Nicole Schlegel, Email = helene.ser...
   

In [86]:
# To get the specific value
print(gsfc.sftgif[0][:][0][0]["x"].values)
print(gsfc.sftgif[0][:][0][0]["y"].values)

-720000.0
-3450000.0


In [105]:
# To get the range
# The way it is structured is, time, y, x 
print(gsfc.sftgif[1][1][0]["x"])

print(gsfc.sftgif[1][1][1])


<xarray.DataArray 'x' ()>
array(-720000.)
Coordinates:
    lon      float64 ...
    lat      float64 ...
    time     object 1981-01-01 00:00:00
    x        float64 -7.2e+05
    y        float64 -3.445e+06
Attributes:
    long_name:  cartesian_x_coordinate
    units:      m
    axis:       X
<xarray.DataArray 'sftgif' ()>
[1 values with dtype=float64]
Coordinates:
    lon      float64 ...
    lat      float64 ...
    time     object 1981-01-01 00:00:00
    x        float64 -7.15e+05
    y        float64 -3.445e+06
Attributes:
    long_name:      land ice area fraction
    standard_name:  land_ice_area_fraction
    units:          1
